# Лабораторная работа №3

ФИО:   
Группа: 

Отправлять можно следующими способами:
1. Запушить этот ноутбук в GitHub в репозиторий, где у вас лежат ноутбуки с лабами

Deadlines:
- Занятие №6 в семестре

Что необходимо сделать:  
- Обучить различного рода модели машинного обучения и сравнить их между собой  

---
## Читайте задание внимательно

Исходные данные:
1. В [табличке](https://docs.google.com/spreadsheets/d/1NOE0D4JQgD6LbvUqWboUI1TFj4P87ugbqUTDquxlGEI/edit?usp=sharing) необходимо узнать название своего датасета 
2. Скачать нужны вам данные можно в [Google Drive](https://drive.google.com/drive/folders/1sbsjBsJ_ln0XgXCI9R6s17pvyvApgcwF?usp=sharing)
  
---
Теперь по пунктам, что я от вас жду:  
1. Загрузить необходимые данные к себе и считать (read) их в переменную.
2. Понять, у вас задача классификации (бинарной или многоклассовой) или регрессии (**если у вас многоклассовая классификация, прочтите P.S.S. внизу**).
3. Сделать предобработку данных:  
     1. Разделить выборку на тренировочную (train) и тестовую (test). _Обратите внимание, что обучать скейлеры и определять, какими значениями вы будете заполнять пропуски, вы будете на train выборке, а применять и на train, и на test_.
     2. Проверить пропуски в данных. Если они есть, заполнить одной из стратегий, предложенных в ноутбуке для семинара №3. P.S. Для численных и категориальных переменных будут разные стратегии.
     3. Отнормировать численные переменные (`StandardScaler`, `MinMaxScaler`).
     4. Закодировать категориальные признаки по одной из стратегий.
4. Обучить на тренировочном множестве:
     1. Линейную модель (`LogisticRegression`, `LinearRegression`)
     2. Деревянную модель (`DecisionTreeClassifier`, `DecisionTreeRegressor`) (тут советую попробовать разные глубины деревьев)
     3. K-ближайших соседей (`KNeighborsClassifier`, `KNeighborsRegressor`) (тут тоже есть смысл попробовать разные `k`)
     4. Случайный лес (`RandomForestClassifier`, `RandomForestRegressor`) 
5. Посчитайте метрики на train и test множествах:
     1. Для задачи классификации -- Accuracy, ROC-AUC (график + значение), PR-кривую (график), F1-score
     2. Для задачи регрессии -- MAE, RMSE, MAPE
6. Сравните метрики относительно train/test, так и относительно разных моделей. Ответьте на следующие вопросы:
     1. Какая модель справилась лучше с поставленной задачей?
     2. Имеет ли место переобучение?
     3. Имеет ли место недообучение?
     4. Как можно улучшить метрики моделей?

---
P.S.  
Просьба -- делать каждое задание в отдельных ячейках и с отдельными заголовками (как пункт 1 и 2 в этом ноутбуке) типа  
- Заголовок
- Ячейки с кодом
- Другой заголовок
- Другие ячейки с кодом

P.S.S.  
Если вам повезло с многоклассовой классификацией, вам будет необходимо понять, умеет ли алгоритм работать с несколькими классами одновременно (обычно они не умеют). Поэтому вам может понадобиться такая штука, как OneVsRestClassifier ([ссылка](https://scikit-learn.org/stable/modules/generated/sklearn.multiclass.OneVsRestClassifier.html#sklearn.multiclass.OneVsRestClassifier)), но советую ознакомиться с этой [страницей](https://scikit-learn.org/stable/modules/multiclass.html), здесь представлена более полная информация.

## 1. Пример импорта данных. Грузим данные

In [49]:
import pandas as pd
data = pd.read_csv("/Users/cene/homework_ai/data/student-por.csv")

## 2. Понимаем, какая перед нами задача

In [50]:
'''
G3 - final grade (numeric: from 0 to 20, output target)
значит, что можно сначала решить регрессией и классификацией, но регрессией будет лучше
'''

'\nG3 - final grade (numeric: from 0 to 20, output target)\nзначит, что можно сначала решить регрессией и классификацией, но регрессией будет лучше\n'

## 3. Делаем предобработку данных

In [51]:
# train_test_split
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

data_object = data.select_dtypes(include = ['object'])

impute = SimpleImputer(strategy = 'most_frequent')
job = impute.fit_transform(data_object[['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic']])

job = pd.DataFrame(job)
ohec = OneHotEncoder(sparse_output=False)
ohec.fit_transform(job)
ohec.categories_
encoded_job = pd.DataFrame(ohec.fit_transform(job), columns=ohec.get_feature_names_out(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic']))
data_int = data.select_dtypes(include=['int64'])
data_new = pd.concat([encoded_job, data_int], axis=1)
X1 = data_new.drop(columns = ['G3', 'G1', 'G2'])
y1 = data_new["G3"]
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size =0.2, random_state=655) 



In [52]:
#isnull
print(data.isnull().sum())

school        0
sex           0
age           0
address       0
famsize       0
Pstatus       0
Medu          0
Fedu          0
Mjob          0
Fjob          0
reason        0
guardian      0
traveltime    0
studytime     0
failures      0
schoolsup     0
famsup        0
paid          0
activities    0
nursery       0
higher        0
internet      0
romantic      0
famrel        0
freetime      0
goout         0
Dalc          0
Walc          0
health        0
absences      0
G1            0
G2            0
G3            0
dtype: int64


In [53]:
from sklearn.preprocessing import StandardScaler
data = [[0, 0], [0, 0], [1, 1], [1, 1]]
scaler = StandardScaler()
scaler.fit(X1_train)
transformed_X1_train = scaler.transform(X1_train)
transformed_X1_test = scaler.transform(X1_test)

## 4 и 5. Обучение и метрики

In [54]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import numpy as np

def print_regression_metrics(y_true, y_pred, model_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100  # в процентах
    
    print(f"{model_name}:")
    print(f"   MAE   : {mae:.4f}")
    print(f"   RMSE  : {rmse:.4f}")
    print(f"   MAPE  : {mape:.2f}%")
    print("-" * 40)
    
    return mae, rmse, mape

In [55]:
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(transformed_X1_train, y1_train)
y_pred_lr = linreg.predict(transformed_X1_test)
mae, rmse, mape = print_regression_metrics(y1_test, y_pred_lr, "LinearRegression")


LinearRegression:
   MAE   : 2.1974
   RMSE  : 8.0145
   MAPE  : 87492410327194688.00%
----------------------------------------


In [56]:
from sklearn.tree import DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=5, random_state=42)
dt.fit(transformed_X1_train, y1_train)
y_pred_dt = dt.predict(transformed_X1_test)
mae, rmse, mape = print_regression_metrics(y1_test, y_pred_dt, "DecisionTreeRegressor")


DecisionTreeRegressor:
   MAE   : 2.5598
   RMSE  : 12.5281
   MAPE  : 32663469824884944.00%
----------------------------------------


In [57]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=8,)
knn.fit(transformed_X1_train, y1_train)
y_pred_knn = knn.predict(transformed_X1_test)
mae, rmse, mape = print_regression_metrics(y1_test, y_pred_knn, "KNeighborsRegressor")


KNeighborsRegressor:
   MAE   : 2.2106
   RMSE  : 8.3840
   MAPE  : 106094414298631872.00%
----------------------------------------


In [58]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(
    n_estimators=100,
    max_depth=5,
    random_state=42,
)
rf.fit(transformed_X1_train, y1_train)
y_pred_rf = rf.predict(transformed_X1_test)
mae, rmse, mape = print_regression_metrics(y1_test, y_pred_rf, "RandomForestRegressor")


RandomForestRegressor:
   MAE   : 2.2733
   RMSE  : 8.1822
   MAPE  : 52093467066605664.00%
----------------------------------------


In [ ]:
'''
лучше всего справился LinearRegression
улучшить можно если еще что нибудь с фичами поделать
'''

'\nлучше всего справился RandomForestRegressor\nулучшить можно если еще что нибудь с фичами поделать\n'